In [128]:
import re
import numpy as np
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from lxml import etree
from multiprocess import Pool, cpu_count
from sqlalchemy import create_engine

In [129]:
db_server='pymysql'
user='root'
db_port = '3306'
password = 'admin'
ip = 'localhost'
db_name = 'animation_movies'
engine = create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()

response = requests.get("https://www.imdb.com/search/title/?title_type=movie&genres=animation&sort=release_date,asc&explore=title_type,genres")
html = response.content
soup = BeautifulSoup(html)
total_titles = int(soup.select("#main > div > div.nav > div.desc > span:nth-child(1)")[0].text.replace("1-50 of ","").replace(",","").replace(" titles.",""))
total_pages = (total_titles//50)+1
page = [(50*p)+1 for p in range(total_pages)]

In [130]:
def extract_urls(page):
    """função vai extrair os links dos filmes em cada página da busca"""
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(f"https://www.imdb.com/search/title/?title_type=movie&genres=animation&sort=release_date,asc&explore=title_type,genres&start={page}")
    html = response.content
    soup = BeautifulSoup(html)
    return list(map(lambda x: "https://www.imdb.com" + x.get("href"), soup.select("div > div > div > h3 > a")))

In [131]:
pool = Pool()
pages = pool.map(extract_urls, page)
pool.terminate()

In [132]:
urls = [url for url_list in pages for url in url_list]

In [133]:
len(urls)

8222

In [134]:
def extract_data(url):
    """função vai extrair os dados dos filmes em cada link"""
    import requests
    from bs4 import BeautifulSoup
    from lxml import etree
    import pandas as pd
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html)
    movie = {}
    movie["url"] = url
    movie["name"] = soup.select("h1")[0].text
    try:
        movie["release year"] = soup.select("a[href$='releaseinfo?ref_=tt_ov_rdat']")[0].text
    except:
        movie["release year"] = "unknown"
    try:
        soup_xpath = etree.HTML(soup.prettify())
        movie["rating"] = soup_xpath.xpath("//*[contains(text(), 'IMDb RATING')]/following-sibling::a/div/div/div/div/span")[0].text.strip()
    except:
        movie["rating"] = "unknown"
    try:
        soup_xpath = etree.HTML(soup.prettify())
        movie["votes"] = soup_xpath.xpath("//*[contains(text(), 'IMDb RATING')]/following-sibling::a/div/div/div/div")[2].text.strip()
    except:
        movie["votes"] = "unknown"
    try:
        movie["country of origin"] = ", ".join(list(map(lambda x: x.text, soup.select("a[href^='/search/title/?country_of_origin=']"))))
    except:
        movie["country of origin"] = "unknown"
    try:
        movie["languages"] = ", ".join(list(map(lambda x: x.text, soup.select("a[href^='/search/title?title_type=feature&primary_language=']"))))
    except:
        movie["languages"] = "unknown"
    try:
        soup_xpath = etree.HTML(soup.prettify())
        director_ = soup_xpath.xpath("//*[contains(text(), 'Director')]/following-sibling::div/ul/li/a")
        movie["director"] = ", ".join(list(set(map(lambda x: x.text.strip(), director_))))
    except:
        movie["director"] = "unknown"
    try:
        soup_xpath = etree.HTML(soup.prettify())
        star_ = soup_xpath.xpath("//*[contains(text(), 'Star')]/following-sibling::div/ul/li/a")
        movie["stars"] = ", ".join(list(set(map(lambda x: x.text.strip(), star_))))
    except:
        movie["stars"] = "unknown"
    movie["html"] = response.text
    return pd.DataFrame(movie, index=[0])

In [135]:
pool = Pool()
data = pool.map(extract_data, urls[0:1000])
pool.terminate()

In [136]:
movies = pd.concat(data)

In [137]:
movies.to_csv("imdb_animation_movies_pt1.csv", sep="|", index=False)

In [138]:
pool = Pool()
data = pool.map(extract_data, urls[1000:3000])
pool.terminate()

In [139]:
movies = pd.concat(data)

In [140]:
movies.to_csv("imdb_animation_movies_pt2.csv", sep="|", index=False)

In [141]:
pool = Pool()
data = pool.map(extract_data, urls[3000:6000])
pool.terminate()

In [142]:
movies = pd.concat(data)

In [143]:
movies.to_csv("imdb_animation_movies_pt3.csv", sep="|", index=False)

In [144]:
pool = Pool()
data = pool.map(extract_data, urls[6000:])
pool.terminate()

In [145]:
movies = pd.concat(data)

In [146]:
movies.to_csv("imdb_animation_movies_pt4.csv", sep="|", index=False)

In [147]:
movies1 = pd.read_csv("imdb_animation_movies_pt1.csv", sep="|")

In [148]:
movies2 = pd.read_csv("imdb_animation_movies_pt2.csv", sep="|")

In [149]:
movies3 = pd.read_csv("imdb_animation_movies_pt3.csv", sep="|")

In [150]:
movies4 = pd.read_csv("imdb_animation_movies_pt4.csv", sep="|")

In [151]:
anim_movies = pd.concat([movies1,movies2,movies3,movies4])

In [196]:
movies = anim_movies.reset_index(drop=True)
movies.drop(columns="html", inplace=True)

In [197]:
movies["country of origin"] = movies["country of origin"].str.split(", ")

In [198]:
movies["languages"] = movies["languages"].str.split(", ")

In [199]:
sep_countries = movies[["url", "country of origin"]]

In [200]:
countries = sep_countries.explode("country of origin")

In [201]:
countries.loc[countries["country of origin"] == "Yugoslavia", "country of origin"] = "Serbia"

In [202]:
countries.loc[countries["country of origin"] == "Serbia and Montenegro", "country of origin"] = "Serbia"

In [203]:
countries.loc[countries["country of origin"] == "Federal Republic of Yugoslavia", "country of origin"] = "Serbia"

In [204]:
countries.loc[countries["country of origin"] == "Soviet Union", "country of origin"] = "Russia"

In [205]:
countries.loc[countries["country of origin"] == "Occupied Palestinian Territory", "country of origin"] = "Palestinian Territories"

In [206]:
countries.loc[countries["country of origin"] == "East Germany", "country of origin"] = "Germany"

In [207]:
countries.loc[countries["country of origin"] == "West Germany", "country of origin"] = "Germany"

In [208]:
countries.loc[countries["country of origin"] == "Czechoslovakia", "country of origin"] = "Czech Republic"

In [209]:
countries.loc[countries["country of origin"] == "Czechia", "country of origin"] = "Czech Republic"

In [210]:
countries.to_sql(name='countries', con=conn, if_exists='replace', index=False)

In [211]:
sep_languages = movies[["url", "languages"]]

In [212]:
languages = sep_languages.explode("languages")

In [213]:
languages.to_sql(name='languages', con=conn, if_exists='replace', index=False)

In [221]:
rating = movies[["url","rating","votes"]]
rating.loc[rating["rating"] == "unknown", "rating"] = np.nan
rating.loc[rating["votes"] == "unknown", "votes"] = np.nan
rating.dropna(subset=["votes"],inplace = True)

C:\Users\Patricia\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Patricia\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Patricia\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [232]:
rating["votes"] = rating["votes"].str.replace("K","*1000")

C:\Users\Patricia\AppData\Local\Temp/ipykernel_12828/971808868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating["votes"] = rating["votes"].str.replace("K","*1000")


In [235]:
rating["votes"] = rating["votes"].str.replace("M","*1000000")

C:\Users\Patricia\AppData\Local\Temp/ipykernel_12828/827333234.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating["votes"] = rating["votes"].str.replace("M","*1000000")


In [237]:
rating["votes"] = rating["votes"].map(pd.eval).astype(int)

C:\Users\Patricia\AppData\Local\Temp/ipykernel_12828/2233325294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating["votes"] = rating["votes"].map(pd.eval).astype(int)


In [238]:
rating.to_sql(name='rating', con=conn, if_exists='replace', index=False)

In [224]:
movies.drop(columns=["country of origin","languages","rating","votes"],inplace=True)

In [225]:
movies.to_sql(name='movies', con=conn, if_exists='replace', index=False)

In [172]:
bechdel = pd.read_csv("raw_bechdel.csv")

In [173]:
bechdel.loc[7595,"imdb_id"] = 810819

In [174]:
bechdel.loc[8209,"imdb_id"] = 451279

In [175]:
bechdel.loc[8547,"imdb_id"] = 2527338

In [176]:
bechdel.dropna(subset=["imdb_id"],inplace = True)

In [177]:
bechdel["imdb_id"] = bechdel["imdb_id"].astype(int).astype(str)

In [178]:
bechdel["imdb_id"] = bechdel["imdb_id"].str.zfill(7)

In [179]:
bechdel["imdb_id"] = "https://www.imdb.com/title/tt" + bechdel["imdb_id"] + "/"

In [180]:
bechdel.to_sql(name='bechdel', con=conn, if_exists='replace', index=False)